In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic

In [2]:
data = pd.read_csv('balanced_data.csv')

In [3]:
tops = data[data['category'] == 'Shirts/Tops']
reviews = tops['text'].tolist()
ratings = tops['rating'].astype(int).tolist()


In [4]:
reviews = [str(i) for i in reviews]

In [5]:
tops.columns

Index(['Unnamed: 0', 'rating', 'parent_asin', 'text', 'title',
       'average_rating', 'category'],
      dtype='object')

In [6]:
#topic_model = BERTopic(language="english", verbose=True)
#topics, probs = topic_model.fit_transform(reviews)

In [7]:
#topic_model.get_topic_info()

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

In [24]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
topic_model = BERTopic(embedding_model=embedding_model)

In [25]:
vectorizer_model = CountVectorizer(
    stop_words="english",
    ngram_range=(1, 2)   
)

In [26]:

topic_model = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model)

In [28]:
topics, probs = topic_model.fit_transform(reviews)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [29]:
tops['topic'] = topics
tops['topic_probs'] = probs

In [45]:
tops_cleaned = tops[tops['topic'] != -1]

# Calculate sentiment polarity for each review
from textblob import TextBlob
import nltk

# Download required NLTK data (only needed first time)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

# Polarity ranges from -1 (negative) to 1 (positive)
sentiment_scores = []
for review in tops_cleaned['text']:
    blob = TextBlob(str(review))
    sentiment_scores.append(blob.sentiment.polarity)

tops_cleaned['sentiment_polarity'] = sentiment_scores

print(f"Sentiment scores calculated. Range: {min(sentiment_scores):.3f} to {max(sentiment_scores):.3f}")
print(f"Mean sentiment: {np.mean(sentiment_scores):.3f}")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/praneethapratapa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/praneethapratapa/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Sentiment scores calculated. Range: -1.000 to 1.000
Mean sentiment: 0.158


In [46]:
tops_cleaned.head()

,Unnamed: 0,rating,parent_asin,text,title,average_rating,category,topic,topic_probs,sentiment_polarity
0,0,2.0,B08NHJ5S5H,Not long enough. I like shirt that cover or be...,Dokotoo Women's Ladies Spring Basic Ribbed Str...,3.6,Shirts/Tops,12,0.757932,0.012500
3,3,3.0,B071F91SCM,"Very pretty, but the post is super thick and t...",925 Sterling Silver Cubic Zirconia Purple Butt...,4.7,Shirts/Tops,17,0.237646,0.119444
9,9,4.0,B08JGGTCB9,I got it for my bf for his birthday was disapp...,Jordan Paris Saint-Germain Long-Sleeve T-Shirt...,4.0,Shirts/Tops,231,1.000000,-0.125000
17,17,3.0,B01IP4GEA2,Small and itchy.,Romwe Women's Striped Crewneck Short Sleeve Lo...,3.3,Shirts/Tops,158,0.908018,-0.250000
61,61,4.0,B01EN4KVRK,Cute little dress but does run on the smaller ...,Tenworld Women Crew Neck Short Sleeve Striped ...,3.4,Shirts/Tops,0,1.000000,0.177083


In [47]:
#topic_model.get_topic_info(1)['Representative_Docs']

print(topic_model.get_topic_info(2)['Representative_Docs'])

0    [Got xl fits like a medium, Ordered a large bu...
Name: Representative_Docs, dtype: object


In [48]:
topic_model.get_representative_docs(45)

["The shirt arrived within 48 hours with prime, however I was so disappointed. The material is rayon/polyester versus cotton and looks like it will melt if you attempt to iron it. Also, the holes have no rhyme or reason and look like someone ways rushing to get the shirt off the press and to the customer so they cut random gignormous holes in different places. There is a gaping hole in the abs section and other unforgiving holes throughout the shirt but not resembling the advertised photo. This a very amateur attempt at making a distressed top as I could've made a better one on my own. I am returning it.",
 'Product is great.  I rarely have anything negative to comment about regarding Amazon but this time I feel I need to offer my opinion - packaging and delivery are top notch 95% of the time.  This delivery came without any outside packaging - in other words, it was delivered in a clear plastic bag for all to see from my front steps.  Granted, the delivery was only a set of big boys P

In [49]:
topic_model.get_topic_info(45)

,Topic,Count,Name,Representation,Representative_Docs
0,45,83,45_delivery_shipping_fast_arrived,"[delivery, shipping, fast, arrived, shipped, q...","[The shirt arrived within 48 hours with prime,..."


In [50]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,15494,-1_shirt_size_material_like,"[shirt, size, material, like, fit, small, fabr...","[It's good, Fabric is almost see through, Not ..."
1,0,1740,0_dress_love dress_dresses_cute dress,"[dress, love dress, dresses, cute dress, pocke...","[LOVE this dress!!!, Love dress, Dress is see ..."
2,1,959,1_tops_cute_love_beautiful,"[tops, cute, love, beautiful, fits, flattering...","[I actually really like this product, but it's..."
3,2,468,2_xl_xxl_medium_large,"[xl, xxl, medium, large, fits like, ordered, o...","[Got xl fits like a medium, Ordered a large bu..."
4,3,357,3_blouse_nice blouse_blouses_cute blouse,"[blouse, nice blouse, blouses, cute blouse, be...","[Really like this blouse., cute blouse., Very ..."
...,...,...,...,...,...
382,381,10,381_soft material_length sized_shorter stated_...,"[soft material, length sized, shorter stated, ...",[Soft and stretchy but not as supportive as I'...
383,382,10,382_shirt awesome_awesome shirt_shirt cute_way...,"[shirt awesome, awesome shirt, shirt cute, way...","[Awesome shirt, AWESOME SHIRT, Awesome shirt]"
384,383,10,383_bigger br_just felt_felt like_little larger,"[bigger br, just felt, felt like, little large...",[I loved it on the Today show and they said it...
385,384,10,384_lbs 5ft_34 110_115 lbs_110,"[lbs 5ft, 34 110, 115 lbs, 110, 115, 5ft, look...",[I bought this and it's cute but once I got it...


#### Training the Model

##### Linear Regression

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Convert topic_probs Series to a 2D numpy array
topic_features = np.vstack(tops_cleaned['topic_probs'].values)

# Add sentiment polarity as an additional feature
sentiment_features = tops_cleaned['sentiment_polarity'].values.reshape(-1, 1)

# Combine topic probabilities with sentiment features
combined_features = np.hstack([topic_features, sentiment_features])

print(f"Feature shape: {combined_features.shape}")
print(f"  - Topic probabilities: {topic_features.shape[1]} features")
print(f"  - Sentiment polarity: 1 feature")

X_train, X_test, y_train, y_test = train_test_split(
    combined_features, tops_cleaned['rating'], test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)

print("MSE:", mse)


Feature shape: (18439, 2)
  - Topic probabilities: 1 features
  - Sentiment polarity: 1 feature
MSE: 1.3886376253463302


In [53]:
#calculate RMSE
rmse = np.sqrt(mse)

print ("RMSE:", rmse)


RMSE: 1.1784046950629186


In [54]:
from sklearn.metrics import r2_score
r2_model = r2_score(y_test, preds)

In [55]:
print (r2_model)

0.2850046990905425


###### We saw that even after removing stopwords, only 0.025% of the variability in our outcome variable is explained by our model (using default BERTopic embedding model all-MiniLM-L6-v2). Learned that sentence‑transformer models (e.g., mpnet, MiniLM, e5) are trained on natural sentences that contain stop words. the R2 improved to 0.165% after using "all-mpnet-base-v2" model. Adding sentiment improved the R2 to 0.285 for linear regression.

#### XGBoost

In [56]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6
)

xgb_model.fit(X_train, y_train)

# Make predictions on test set
xgb_preds = xgb_model.predict(X_test)

# Calculate evaluation metrics
xgb_mse = mean_squared_error(y_test, xgb_preds)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_preds)

print("XGBoost Model Performance:")
print(f"MSE: {xgb_mse:.4f}")
print(f"RMSE: {xgb_rmse:.4f}")
print(f"R² Score: {xgb_r2:.4f}")


XGBoost Model Performance:
MSE: 1.3557
RMSE: 1.1643
R² Score: 0.3020
